# Import packages

In [1]:
import os
import json
import numpy as np
from sympy import symbols, sympify, lambdify
from tabulate import tabulate
import yaml
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import NonNeg

# Set random seeds for reproducibility
seed_value = 2023
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

2023-11-13 16:24:26.063351: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-13 16:24:26.065999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 16:24:26.105709: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 16:24:26.105743: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 16:24:26.105827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

# Constants and parameters

In [2]:
with open('config_run.yml', 'r') as ymlfile:
    config_data = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [3]:
config_data

{'problem_folder': 'ERK_equations_s2p2',
 'load_initial_estimate': 'best_estimate_231113_161613',
 'save_best_solution': True,
 'epochs': 501}

In [4]:
problem_name = config_data['problem_folder']
EQS_FILE = problem_name + '.json'
EQS_PATH = os.path.join(os.getcwd(), problem_name, EQS_FILE)


training_steps = config_data['epochs']
display_step = training_steps // 10

load_initial_estimate = config_data['load_initial_estimate']
save_best_solution = config_data['save_best_solution']

# learning_rate = 1e-2
learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 2],[1e-2, 1e-3])
# learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 3, 2 * training_steps // 3],[1e-2,5e-3,1e-3])
# learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay([training_steps // 3, 2 * training_steps // 3],[5e-2,1e-2,5e-3])

# Load Equations

In [5]:
# Read the JSON data from the file
with open(EQS_PATH, 'r') as file:
    data = json.load(file)

variables = data['variables']
equations = data['equations']
# equation_terms = data['equation_terms']

variables_str = ' '.join(variables)
variables_sym = symbols(variables_str)
equations_sym = [sympify(eq) for eq in equations]  

## Identify the equation terms
equation_terms = []
for (i, eq) in enumerate(equations_sym):
    terms = []
    for term in eq.args:
        terms.append(str(term))
    equation_terms.append(terms)

# Initialize a list to store the coefficients
scalar_coefficients = []
functions = []
bias_coefficients = []

# Loop through the equation terms
for terms in equation_terms:
    bias_coefficients.append(float(terms[0]))
    coefficients = []   
    for term in terms[1:]:
        # Split each term by the first '*'
        term_parts = term.split('*')
        # Extract the scalar coefficient or default to '1'
        scalar_coeff = term_parts[0] if len(term_parts) > 1 and term_parts[0] not in variables else '1.0'
        coefficients.append(float(scalar_coeff))
               
        # Remove first occurrence of scalar coefficient from the term
        func_coeff = term.replace(term_parts[0]+'*', '', 1) if len(term_parts) > 1 and term_parts[0] not in variables else term
        functions.append(func_coeff)
        
    scalar_coefficients.append(coefficients)   
       
functions_sym = [sympify(func) for func in functions]       
        
print(bias_coefficients)
print(scalar_coefficients)
print(functions)
print(functions_sym)
 


[-1.0, -1.0]
[[1.0, 1.0], [2.0]]
['b_1', 'b_2', 'a_21*b_2']
[b_1, b_2, a_21*b_2]


# Model

## Create model

In [6]:
# Network Parameters
num_input = 1 # input layer

num_vars = len(variables)
num_eqs = len(equations)
num_functions = len(functions)

num_hidden = [num_vars, num_functions]
num_output = num_eqs # output layer

display(
    f'num_hidden = {num_hidden}',
    f'num_output = {num_output}'
)

'num_hidden = [3, 3]'

'num_output = 2'

In [7]:
## Weights to Layer 3 - determined by the functions contains the variables
w23_flags = [[True] * num_functions for _ in range(num_vars)]
for vi, var in enumerate(variables):
    for fi, func in enumerate(functions):
        # print(f'var = {var}, func = {func}', var not in func)
        if var not in func:
            w23_flags[vi][fi] = False

w23_flags = tf.constant(w23_flags, dtype=tf.bool)

# Initialize the weights (w23) with zeros
w23 = tf.constant(tf.zeros(num_hidden, dtype=tf.float32))
# Set the weights to 1 where func_flags is True
w23 = tf.where(w23_flags, tf.ones_like(w23), w23)
w23 = tf.transpose(w23)

## Weights to Layer 4 - determined by the coefficients of the functions
# Initialize the weights (w34) with zeros
w34_np = np.zeros([num_functions, num_output], dtype=np.float32)
# Assign scalar coefficients to the first column of w34_np
row = 0
for sci, scalar_coeffs in enumerate(scalar_coefficients):
    for scj, scl_coeff in enumerate(scalar_coeffs):
        w34_np[row, sci] = scl_coeff
        row += 1
# Tranform to tensor
w34 = tf.constant(w34_np, dtype=tf.float32)

## IC for classic methods
# w12_rk2 = tf.Variable([[0.8, 0.3, 0.3]])
# w12_rk4 = tf.Variable([[0.3, -0.1, 0.63, 0.2, 0.1, 0.8, 0.1, 0.2, 0.4, 0.1]])
# w12_r_s5p5 = tf.Variable([[0.35030925,  0.0, 0.43791163, 0.07676958, 0.0, 0.6589101,
#                             0.21143106, 0.31450838, 0.0, 0.34649932, 0.17551975, 0.23256625,
#                             0.18079841, 0.17328405, 0.23783126]], dtype=tf.float32,  constraint=NonNeg())
# w12_vicky = tf.Variable([[0.0, 0.0, 0.0, 1.5, 0.0, 0.0, 0.8, 0.8, 0.0, 0.8, 0.0, 0.9, 0.2, 0.0, 0.0, 1.0]],
#                 dtype=tf.float32, 
#                 constraint=NonNeg())

if load_initial_estimate is None:
    w12 =  tf.Variable(tf.random.normal([num_input, num_hidden[0]]), 
                  dtype=tf.float32, 
                  constraint=NonNeg())
    
    initial_solution = w12.numpy()[0].tolist()
    
else:
    print('Loading initial estimate...')
    # Load json file with initial estimate (best from previous run)
    with open(os.path.join(problem_name, f'{load_initial_estimate}.json'), 'r') as file:
        data = json.load(file)
    init_estimate = data['best_solution']
    initial_solution = [float(x) for x in init_estimate.values()]
    w12 = tf.Variable([initial_solution], dtype=tf.float32, constraint=NonNeg())
    
    
# Store layers weight & bias
weights = {
    ## Variables - weights to Layer 1
    # Create the variable with the non-negativity constraint
    'w12': w12,
    # Whether the functions in F1 and F2 contain the variables x1 and x2
    'w23': w23,
    # # The coefficients of the functions in F1 and F2
    'w34': w34
}

biases = {
    'b12': tf.constant(tf.zeros([num_hidden[0]], dtype=tf.float32)),
    'b23': tf.constant(tf.zeros([num_hidden[1]], dtype=tf.float32)),
    'out': tf.constant([bias_coefficients], dtype=tf.float32),
}

# Stochastic gradient descent optimizer.
optimizer = Adam(learning_rate=learning_rate, name='custom_optimizer_name')

Loading initial estimate...


In [8]:
weights['w12']

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32, numpy=array([[0.7293615 , 0.31465805, 0.6854762 ]], dtype=float32)>

## Model functions

In [9]:

def evaluate_tf_function(inputs, values, symbolic_function):
    # Ensure that the number of inputs and values match
    if len(inputs) != len(values):
        raise ValueError("Number of inputs and values must match.")

    # Evaluate the symbolic function using TensorFlow and the provided values
    result = symbolic_function(*values)
    # Convert the result to a TensorFlow tensor
    result = tf.convert_to_tensor(result, dtype=tf.float32)
    
    # print(result.__class__)
    
    return result

# Define the custom activation function with @tf.function
# @tf.function
def activation_layer2(layer, vars=variables_sym, funcs=functions_sym):
    
    layer_values = layer.numpy()[0]
    # var_vals = [layer_values[i] for i in range(layer_values.shape[0])]
    ##x1_val = tf.squeeze(layer[0, 0])
    var_vals = [tf.squeeze(layer[0, i]) for i in range(layer_values.shape[0])]
    
    # print(f'var_vals = {var_vals}')
   
    # var_vals_dict = dict(zip(vars, var_vals))
    # print(f'var_vals_dict = {var_vals_dict}')
    # # Step 3 and 4: Substitute values into the functions and evaluate
    # layer_act = [func.subs(var_vals_dict).evalf() for func in funcs]
    # # Convert SymPy Float to Python float
    # layer_act = [tf.convert_to_tensor(float(val.evalf()), dtype=tf.float32) for val in layer_act]
    
    layer_act = list()
    for func in funcs:
        func_tf = lambdify(vars, func, 'tensorflow')
        layer_act.append(evaluate_tf_function(variables_sym, var_vals, func_tf))
        # result = func_tf(*var_vals)
        # layer_act.append(result)
    
    # print(f'layer_act = {layer_act}')
    
    # layer_2 = tf.stack(layer_act, axis=0)  # Stack the list of tensors into a single tensor
    # layer_2 = tf.reshape(layer_2, [1, len(funcs)])
    
    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf.float32)
    
    # print(type(layer_2))
    
    # print(f'layer_2 = {layer_2}')
    
    layer_2 = tf.reshape(layer_2, [1, len(funcs)])

    return layer_2


# Create model
def multilayer_perceptron(weights, biases):

    # # Reshape input if necessary, matching the shape of the first layer's weights
    # x = tf.reshape(x, [1, -1])  # Adjust the shape as needed

    # layer_1 = tf.add(tf.matmul(x, weights['w12']), biases['b12'])
    # layer_1 = tf.matmul(x, weights['w12'])
    layer_1 = weights['w12']
    
    # print(f'layer_1 = {layer_1.shape}')

    layer_2 = activation_layer2(layer_1)
    
    # Output fully connected layer
    output = tf.add(tf.matmul(layer_2, weights['w34']), biases['out'])
    
    # # Add RELU activation function to output layer
    # output = tf.nn.relu(output)
    
    return output, layer_1


def loss_function(weights, biases):
    
    output, _ = multilayer_perceptron(weights, biases)

    return tf.reduce_mean(tf.square(output))


# Train step
def train_step(weights, biases, optimizer):

    with tf.GradientTape() as tape:
        
        loss = loss_function(weights, biases)

    trainable_variables = [weights['w12']]  # list containing only 'w12'
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss    




## Train model

In [10]:
best_loss = np.inf  # Initialize with infinity or an appropriate initial value

# Initialize variables to store the best solution
best_solution = None
best_epoch = None
# # Define the number of epochs after which to save the best weights
# save_interval = training_steps // 100    # Every 1% of the training steps 

for i in range(training_steps):
    
    current_loss = train_step(weights, biases, optimizer)
    
    # Check if the current loss is better than the best loss
    if current_loss < best_loss:
        best_loss = current_loss
        best_epoch = i
        best_solution = weights['w12'].numpy()[0].copy()
       
    if i % display_step == 0:
        print(f"epoch {i:6d} => loss: {current_loss:.16e} | best loss: {best_loss:.16e} ({best_epoch})")
        
    if current_loss <= 1e-16:
        print(f"epoch {i:6d} => loss: {current_loss:.16e} | best loss: {best_loss:.16e} ({best_epoch})")

epoch      0 => loss: 1.2222274747841766e-08 | best loss: 1.2222274747841766e-08 (0)
epoch     50 => loss: 6.3245090586860897e-07 | best loss: 1.2222274747841766e-08 (0)
epoch    100 => loss: 4.4951846689400554e-09 | best loss: 3.5818459309666650e-10 (97)
epoch    150 => loss: 3.0155433705658652e-11 | best loss: 1.0302869668521453e-12 (144)
epoch    200 => loss: 1.7763568394002505e-13 | best loss: 1.4210854715202004e-14 (198)
epoch    220 => loss: 0.0000000000000000e+00 | best loss: 0.0000000000000000e+00 (220)
epoch    250 => loss: 3.5527136788005009e-14 | best loss: 0.0000000000000000e+00 (220)
epoch    258 => loss: 0.0000000000000000e+00 | best loss: 0.0000000000000000e+00 (220)
epoch    275 => loss: 0.0000000000000000e+00 | best loss: 0.0000000000000000e+00 (220)
epoch    280 => loss: 0.0000000000000000e+00 | best loss: 0.0000000000000000e+00 (220)
epoch    289 => loss: 0.0000000000000000e+00 | best loss: 0.0000000000000000e+00 (220)
epoch    294 => loss: 0.0000000000000000e+00 | b

## Save best solution

In [11]:
if save_best_solution is not None:
    
    best_solution_dict = dict()
    initial_solution_dict = dict()
    for vi, var in enumerate(variables):
        initial_solution_dict[var] = f'{initial_solution[vi]:.16f}'
        best_solution_dict[var] = f'{best_solution[vi]:.16f}'
        
    save_solution_dict = {
        'initial_estimate': initial_solution_dict,
        'best_solution': best_solution_dict,
        'best_epoch': best_epoch,
    }
        
    now = datetime.now()
    dt_string = now.strftime("%y%m%d %H%M%S")
    save_file_path = os.path.join(problem_name, f'best_estimate_{dt_string.split()[0]}_{dt_string.split()[1]}.json')
    
    # Save the configuration data to the JSON file
    with open(save_file_path, 'w') as config_file:
        json.dump(save_solution_dict, config_file, indent=4)

In [12]:
best_solution

array([0.7287481, 0.3138917, 0.6861083], dtype=float32)

In [13]:
func_res_model, solution  = multilayer_perceptron(weights, biases)
solution = list(solution.numpy()[0])
best_solution = list(best_solution)

print('BEST SOLUTION')
for i, var in enumerate(variables):
    print(f'{var} = {best_solution[i]:.16f}')
    
print()
print('RESIDUALS')
headers = ['Equation', 'Model residual', 'Function residual']
table_data = []

for f, func in enumerate(equations):
    eq = equations_sym[f]
    equation = lambdify(variables_sym, eq)
    func_res_eq = abs(equation(*best_solution))
    table_data.append([f'{func} = ', f'{abs(func_res_model[0, f]):.4e}', f'{func_res_eq:.4e}'])

print(tabulate(table_data, headers=headers, tablefmt='grid'))

BEST SOLUTION
a_21 = 0.7287480831146240
b_1 = 0.3138917088508606
b_2 = 0.6861082911491394

RESIDUALS
+------------------+------------------+---------------------+
| Equation         |   Model residual |   Function residual |
+==================+==================+=====================+
| b_1 + b_2 - 1 =  |                0 |          0          |
+------------------+------------------+---------------------+
| 2*a_21*b_2 - 1 = |                0 |          2.0397e-07 |
+------------------+------------------+---------------------+
